In [12]:
import skmob
import pandas as pd
import csv, os, sys, json
import urllib.request
from skmob.preprocessing import clustering


ModuleNotFoundError: No module named 'skmob'

In [ ]:
def coords_to_place(x, y):
    client_id = "ktta1uumf3"
    client_secret = "PDFJ27DxqU6bQSsf40GE9DwybUnlgRzZnni42Dqi"
    coords = ','.join([x, y])
    url = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?request=coordsToaddr&coords=" + \
        coords + "&sourcecrs=epsg:4326&output=json&orders=roadaddr"
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
    request.add_header("X-NCP-APIGW-API-KEY", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if (rescode == 200):
        response_body = response.read()
        addr_info = response_body.decode('utf-8')
        addr_info = json.loads(addr_info)
    else:
        print("Error Code:" + rescode)

    results = addr_info['results'][0]
    land = results['land']
    building_name = land['addition0']['value']
    road_name = land['name']
    number1 = land['number1']
    number2 = land['number2']

    # search = ' '.join([building_name, road_name, number1, number2])
    search = building_name

    # print(f"building name: {building_name} road addr: {road_name} {number1} {number2}")
    # print(search)
    return search

In [ ]:
def place_to_category(place):
    client_id = "G3_TXQoFDd0lBFsM8fpG"
    client_secret = "oJaqz0CK0W"
    encText = urllib.parse.quote(place)
    url = "https://openapi.naver.com/v1/search/local?query=" + encText  # JSON 결과
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if (rescode == 200):
        response_body = response.read()
        category_info = response_body.decode('utf-8')
        category_info = json.loads(category_info)
    else:
        print("Error Code:" + rescode)

    item = category_info['items'][0]
    category = item['category']
    # print(category)
    return category

In [ ]:
device_id_list = []
for it in os.scandir('./staypoint'):
    if it.is_dir():
        if it.name.isdigit():
            device_id_list.append(it.name)
device_id_list.sort()


def get_week(device_id):
    stdf = skmob.TrajDataFrame.from_file('./staypoint/'+device_id+'/stay_points.csv',
                                         latitude='latitude', longitude='longitude', user_id='deviceid', datetime='arrive_time')
    stdf['leaving_datetime'] = pd.to_datetime(stdf['leaving_datetime'])
    weeks = [g for n, g in stdf.set_index(
        'datetime').groupby(pd.Grouper(freq='W'))]
    return weeks





def get_cluster(weeks):
    cluster_list = list()
    for x in weeks:
        cstdf = clustering.cluster(
            x.reset_index(), cluster_radius_km=0.1, min_samples=1)
        cluster_list.append(cstdf)
    return cluster_list



def get_rep_cluster(cluster_list):
    rep_cluster_list = []
    for df in cluster_list:
        idx = df[df['cluster'] == 0].index
        ndf = df.drop(idx)
        ndf['year'] = ndf['datetime'].map(lambda x: x.isocalendar()[0])
        ndf['week'] = ndf['datetime'].map(lambda x: x.isocalendar()[1])

        # 평균값을 취해서 cluster별로 좌표 추출
        ndf = ndf.groupby('cluster').mean()
        ndf['year'] = ndf['year'].astype('object')
        ndf['category'] = ''

        for idx in ndf.index:
            x = "{:.7f}".format(ndf.loc[idx, 'lng'])
            y = "{:.7f}".format(ndf.loc[idx, 'lat'])
            # print(f"x: {x} y: {y}")
            place = coords_to_place(x, y)
            category = place_to_category(place)
            ndf.loc[idx, 'category'] = category

        rep_cluster_list.append(ndf)

    return rep_cluster_list


for device_id in device_id_list:
    fields = ['deviceid', 'latitude', 'longitude', 'year', 'week', 'category']
    path = "./staypoint/" + device_id
    weeks = get_week(device_id)
    cluster_list = get_cluster(weeks)
    represent = get_rep_cluster(cluster_list)

    with open("./staypoint/"+device_id+'/category.csv', 'w', newline='') as f:
        write = csv.writer(f)
        write.writerow(fields)

        for df in represent:
            rep = df.values.tolist()
            # print(rep)
            write.writerows(rep)


[[100.0, 37.57098826, 127.03692836, 2019.0, 45.0, ''], [100.0, 37.564182, 127.035261, 2019.0, 45.0, ''], [100.0, 37.5571771, 127.02857195, 2019.0, 45.0, ''], [100.0, 37.555773900000005, 127.02371215, 2019.0, 45.0, '']]
[[100.0, 37.55804580833333, 127.03554640833333, 2019.0, 46.0, ''], [100.0, 37.574517900000004, 127.03918365, 2019.0, 46.0, ''], [100.0, 37.322905, 127.9599752, 2019.0, 46.0, ''], [100.0, 37.2984134, 127.6542364, 2019.0, 46.0, ''], [100.0, 37.5718741, 127.0352059, 2019.0, 46.0, '']]
[[103.0, 37.58565438, 126.94953267000001, 2019.0, 45.0, ''], [103.0, 37.53928945, 127.12688455, 2019.0, 45.0, ''], [103.0, 37.517388600000004, 127.0706049, 2019.0, 45.0, ''], [103.0, 37.516333575000004, 127.070962175, 2019.0, 45.0, ''], [103.0, 37.56816225, 127.02315765, 2019.0, 45.0, ''], [103.0, 37.53656625, 127.12546115, 2019.0, 45.0, ''], [103.0, 37.5381098, 127.12326445, 2019.0, 45.0, ''], [103.0, 37.5668039, 126.9919643, 2019.0, 45.0, ''], [103.0, 37.5825383, 126.9504543, 2019.0, 45.0, '

KeyError: 'year'